In [1]:
# Step 1: Download the ZIP file from Google Drive using gdown
!gdown --id 1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3

# Step 2: Unzip the dataset
import zipfile
with zipfile.ZipFile('/content/kidney-ct-scan-image.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

# Step 3: Check the folder structure (optional)
import os
for root, dirs, files in os.walk('/content/dataset'):
    level = root.replace('/content/dataset', '').count(os.sep)
    indent = ' ' * 4 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 4 * (level + 1)
    for f in files[:5]:  # show only first 5 files per folder
        print(f'{subindent}{f}')
    if len(files) > 5:
        print(f'{subindent}... ({len(files)} files)')

# Step 4: Train model
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Parameters
data_dir = "/content/dataset/kidney-ct-scan-image"
image_size = (224, 224)
batch_size = 16
epochs = 15

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Load and freeze base model
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
for layer in base_model.layers[:10]:
    layer.trainable = False

# Add classification head
x = Flatten()(base_model.output)
x = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)

# Compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history=model.fit(train_generator, validation_data=val_generator, epochs=epochs)

# Save model as model.h5
model.save("model.h5")




/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=04e24c78-553a-40d8-afb1-d653e508bb40
To: /content/kidney-ct-scan-image.zip
100% 57.7M/57.7M [00:00<00:00, 90.2MB/s]
dataset/
    kidney-ct-scan-image/
        Tumor/
            Tumor- (960).jpg
            Tumor- (954).jpg
            Tumor- (813).jpg
            Tumor- (837).jpg
            Tumor- (742).jpg
            ... (225 files)
        Normal/
            Normal- (981).jpg
            Normal- (699).jpg
            Normal- (670).jpg
            Normal- (823).jpg
            Normal- (751).jpg
            ... (240 files)
Found 372 images belonging to 2 classes.
Found 93 

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 36s 892ms/step - accuracy: 0.5169 - loss: 5.1301 - val_accuracy: 0.5161 - val_loss: 0.6931
Epoch 2/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 313ms/step - accuracy: 0.4514 - loss: 0.7423 - val_accuracy: 0.5161 - val_loss: 0.6931
Epoch 3/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 405ms/step - accuracy: 0.4209 - loss: 0.7598 - val_accuracy: 0.5161 - val_loss: 0.6943
Epoch 4/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 334ms/step - accuracy: 0.5281 - loss: 0.7024 - val_accuracy: 0.5161 - val_loss: 0.6933
Epoch 5/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 9s 355ms/step - accuracy: 0.5162 - loss: 0.6925 - val_accuracy: 0.5161 - val_loss: 0.6931
Epoch 6/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 334ms/step - accuracy: 0.5125 - loss: 0.7088 - val_accuracy: 0.5161 - val_loss: 0.6927
Epoch 7/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 8s 332ms/step - accuracy: 0.4551 - loss: 0.6963 - val_accuracy: 0.5161 - val_loss: 0.6931
Epoch 8/15
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 302ms/step - accuracy: 0.5233 - loss: 0.6931 - val_accuracy: 0

In [2]:
import tensorflow as tf

# Load your Keras model
model = tf.keras.models.load_model("model.h5")

# Convert to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model
with open("model.tflite", "wb") as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpk2oh4bed'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  132798742300432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132798742304656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132798742299472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132798742306192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132798742306000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132798742306960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132798742306384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132798742307728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132798742307152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132798742308496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132798742307920

In [3]:
print(train_generator.class_indices)


{'Normal': 0, 'Tumor': 1}


In [4]:
# Final loss and accuracy
train_loss = history.history['loss'][-1]
train_acc = history.history['accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_acc = history.history['val_accuracy'][-1]

print(f"\n📊 Final Training Loss: {train_loss:.4f}")
print(f"✅ Final Training Accuracy: {train_acc:.4f}")
print(f"\n📉 Final Validation Loss: {val_loss:.4f}")
print(f"🎯 Final Validation Accuracy: {val_acc:.4f}")



📊 Final Training Loss: 0.6928
✅ Final Training Accuracy: 0.5161

📉 Final Validation Loss: 0.6927
🎯 Final Validation Accuracy: 0.5161
